In [20]:
import pandas as pd
import json
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

In [14]:
# Define the file paths
file_paths = [
    'C:/Users/Mads/Documents/NLP/cnn_dailymail/3.0.0/train-00000-of-00003.parquet',
    'C:/Users/Mads/Documents/NLP/cnn_dailymail/3.0.0/train-00001-of-00003.parquet',
    'C:/Users/Mads/Documents/NLP/cnn_dailymail/3.0.0/train-00002-of-00003.parquet',
]

# Load and concatenate the Parquet files
train_df_list = [pd.read_parquet(file) for file in file_paths]
train_df = pd.concat(train_df_list, ignore_index=True)

# Randomly sample 150,000 rows
sampled_df = train_df.sample(n=1000, random_state=1)

# Display the shape and the first few rows of the sampled DataFrame
print(sampled_df.shape)
print(sampled_df.head())


(1000, 3)
                                                  article  \
121639  By . Daily Mail Reporter . PUBLISHED: . 07:34 ...   
188289  By . Lizzie Parry for MailOnline . A young can...   
75440   Rebels aided by al Qaeda-linked militants seiz...   
49582   (CNN) -- Two rights groups launched a stinging...   
50612   (CNN) -- Puffing on electronic cigarettes is a...   

                                               highlights  \
121639  President Barack Obama's top national security...   
188289  Sophie Walton was diagnosed with bone cancer i...   
75440   The Israeli military has closed off the area a...   
49582   NEW: Trial resumes of Kremlin critic Aleksei N...   
50612   Government proposes to explicitly ban the use ...   

                                              id  
121639  1211a23db42e7dd52a43e564a899ebbf1b2d1251  
188289  aff92aa2fc6e294efeeed70e9b0a0aefac32b030  
75440   bb7757b7bf3d90ec4f1d1915a4699eecafc25d1a  
49582   8f25b2c54cb563b98e1f2619401df7137a3f0c77  
5

In [15]:
sampled_df.to_csv('sampled_df.csv', index=False)

In [21]:
lemmatizer = WordNetLemmatizer()

file_path = 'data.csv'
df = pd.read_csv(file_path)

# Define stop words
stop_words = set(stopwords.words('english'))

# Function to clean text
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Remove punctuation and special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize text
    words = word_tokenize(text)
    
    # Remove stop words and lemmatize
    cleaned_text = ' '.join([lemmatizer.lemmatize(word) for word in words if word not in stop_words])
    
    return cleaned_text

# Apply the cleaning function with progress bars
tqdm.pandas()

In [22]:
# Apply the cleaning function to both the 'article' and 'highlights' columns
df['cleaned_article'] = df['article'].progress_apply(clean_text)

100%|██████████| 150000/150000 [08:42<00:00, 287.04it/s]


In [23]:
df['cleaned_highlights'] = df['highlights'].progress_apply(clean_text)

  0%|          | 0/150000 [00:00<?, ?it/s]C:\Users\Mads\AppData\Local\Temp\ipykernel_11968\3075676461.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
100%|██████████| 150000/150000 [01:09<00:00, 2144.44it/s]


In [29]:
cleaned_file_path = 'cleaned_data.csv'

In [28]:
df.to_csv(cleaned_file_path, index=False)

In [27]:
df.head(10)

,id,cleaned_article,cleaned_highlights
0,1211a23db42e7dd52a43e564a899ebbf1b2d1251,daily mail reporter published est june updated...,president barack obamas top national security ...
1,aff92aa2fc6e294efeeed70e9b0a0aefac32b030,lizzie parry mailonline young cancer victim to...,sophie walton diagnosed bone cancer discoverin...
2,bb7757b7bf3d90ec4f1d1915a4699eecafc25d1a,rebel aided al qaedalinked militant seized con...,israeli military closed area around crossing o...
3,8f25b2c54cb563b98e1f2619401df7137a3f0c77,cnn two right group launched stinging critique...,new trial resume kremlin critic aleksei navaln...
4,daa6ca8fe860fa70bed00e2bfd05ab7414b7cb0e,cnn puffing electronic cigarette already nono ...,government proposes explicitly ban use electro...
5,c0cdf7175fc2032b547fafa8fb72c104886d38b7,barely half century since part thames declared...,million openair pool built middle thames centr...
6,efed3617e3c01ba45b090f6fc6a62a7e77131576,go history year social medium truly came selfi...,oscar selfie retweeted two million time broke ...
7,694eb1e7c38a77d5eabb0e630e4194f47fa0bb08,jailed grandmotheroftwo carla buckle sentenced...,carla buckle taken court paying council tax gr...
8,34964187fef7eb394d7d192f6197d315fcb7d852,baghdad cnn iraqi security force raided farm b...,new curfew imposed anbar province overnight st...
9,e1cffc79b05bd0a1a63137086ede6811762f3fc9,cnn china rallying around fallen hurdler liu x...,china rallying around hurdler liu xiang failed...
